In [ ]:
import torch
import pickle
import warnings
import numpy as np
from tqdm.auto import tqdm
from scipy import spatial
from scipy.io import loadmat
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

warnings.filterwarnings("ignore")


In [ ]:
def pairwise_accuracy(actual, predicted):
    true = 0
    total = 0
    for i in range(0, len(actual)):
        for j in range(i + 1, len(actual)):
            total += 1

            s1 = actual[i]
            s2 = actual[j]
            b1 = predicted[i]
            b2 = predicted[j]

            result1 = spatial.distance.cosine(s1, b1)
            result2 = spatial.distance.cosine(s2, b2)
            result3 = spatial.distance.cosine(s1, b2)
            result4 = spatial.distance.cosine(s2, b1)

            if result1 + result2 < result3 + result4:
                true += 1

    return true / total


In [ ]:
def pearcorr(actual, predicted):
    corr = []
    for i in range(0, len(actual)):
        corr.append(np.corrcoef(actual[i], predicted[i])[0][1])
    return np.mean(corr)


In [ ]:
def generate_indices(data):
    Taskindices = []
    for j in data["meta"][0][0][11][0][5]:
        for k in j[0]:
            Taskindices.append(int(k))
    DMNindices = []
    for j in data["meta"][0][0][11][0][6]:
        for k in j[0]:
            DMNindices.append(int(k))
    Visualindices = []
    Visualindices_body = []
    Visualindices_face = []
    Visualindices_object = []
    Visualindices_scene = []
    for j in data["meta"][0][0][11][0][9]:
        for k in j[0]:
            Visualindices_body.append(int(k))
    for j in data["meta"][0][0][11][0][10]:
        for k in j[0]:
            Visualindices_face.append(int(k))
    for j in data["meta"][0][0][11][0][11]:
        for k in j[0]:
            Visualindices_object.append(int(k))
    for j in data["meta"][0][0][11][0][12]:
        for k in j[0]:
            Visualindices_scene.append(int(k))

    for j in data["meta"][0][0][11][0][13]:
        for k in j[0]:
            Visualindices.append(int(k))
    Languageindices_lh = []
    Languageindices_rh = []
    for j in data["meta"][0][0][11][0][7]:
        for k in j[0]:
            Languageindices_lh.append(int(k))
    for j in data["meta"][0][0][11][0][8]:
        for k in j[0]:
            Languageindices_rh.append(int(k))
    return (
        Taskindices,
        DMNindices,
        Visualindices_body,
        Visualindices_face,
        Visualindices_object,
        Visualindices_scene,
        Visualindices,
        Languageindices_lh,
        Languageindices_rh,
    )


In [ ]:
TASKS = [
    "bert",
    "coref",
    "ner",
    "nli",
    "paraphrase",
    "qa",
    "sa",
    "srl",
    "ss",
    "sum",
    "wsd",
]
ROIS = [
    "language_lh",
    "language_rh",
    "vision_body",
    "vision_face",
    "vision_object",
    "vision_scene",
    "vision",
    "dmn",
    "task",
]
subjects = ["P01", "M02", "M04", "M07", "M15"]
layers_bert = [
    "block1",
    "block2",
    "block3",
    "block4",
    "block5",
    "block6",
    "block7",
    "block8",
    "block9",
    "block10",
    "block11",
    "block12",
    "fc",
]
layers_bart = ["fc"]


In [ ]:
def get_subject_data(subject):
    data_pic1 = loadmat("./pereira_dataset/" + subject + "/data_384sentences.mat")
    data_pic2 = loadmat("./pereira_dataset/" + subject + "/data_243sentences.mat")
    data_pic = loadmat("./pereira_dataset/" + subject + "/data_384sentences.mat")

    data_pic["examples_passagesentences"] = np.concatenate(
        (
            data_pic1["examples_passagesentences"],
            data_pic2["examples_passagesentences"],
        ),
        axis=0,
    )
    data_pic["meta"] = np.concatenate((data_pic1["meta"], data_pic2["meta"]), axis=0)
    (
        Taskindices,
        DMNindices,
        Visualindices_body,
        Visualindices_face,
        Visualindices_object,
        Visualindices_scene,
        Visualindices,
        Languageindices_lh,
        Languageindices_rh,
    ) = generate_indices(data_pic)

    roi_indices = {
        "language_lh": Languageindices_lh,
        "language_rh": Languageindices_rh,
        "vision_body": Visualindices_body,
        "vision_face": Visualindices_face,
        "vision_object": Visualindices_object,
        "vision_scene": Visualindices_scene,
        "vision": Visualindices,
        "dmn": DMNindices,
        "task": Taskindices,
    }

    fmri = {}
    for roi, indices in roi_indices.items():
        fmri[roi] = data_pic["examples_passagesentences"][0:, np.array(indices) - 1]

    return fmri


In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.w = torch.nn.Parameter(torch.randn(11, 1))

    def forward(self, model_outputs, x):
        model_outputs = model_outputs.permute(0, 2, 1)
        weights = self.w**2
        weights = weights / weights.sum()
        weighted_embedding = (model_outputs @ weights).squeeze()
        loss = torch.nn.functional.mse_loss(weighted_embedding, x)
        return loss


In [ ]:
class Custom_Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        super(Custom_Dataset, self).__init__()
        self.X = X
        self.y = y

    def __getitem__(self, idx):
        return (self.X[idx], self.y[idx])

    def __len__(self):
        return len(self.X)


In [ ]:
def get_weights(X_train, y_train):
    train_dataset = Custom_Dataset(X_train, y_train)
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=16, shuffle=True, num_workers=6
    )

    EPOCHS = 100
    model = Net()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in tqdm(range(EPOCHS)):
        losses = []
        for X, y in train_dataloader:
            loss = model(X.float(), y.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        mean_loss = np.mean(losses)

    weights = model.w.detach().numpy().squeeze()
    weights = weights**2
    weights = weights / weights.sum()

    del train_dataset, train_dataloader, model, optimizer, losses, mean_loss

    return weights


def create_ensemble(outputs, voxels):
    outputs = outputs.transpose(1, 0, 2)
    X_train, X_test, y_train, y_test = train_test_split(
        outputs, voxels, test_size=0.2, random_state=42
    )
    weights = get_weights(X_train, y_train)

    X_test = X_test.transpose(0, 2, 1)
    y_pred = X_test @ weights
    acc = pairwise_accuracy(y_test, y_pred)
    corr = pearcorr(y_test, y_pred)
    del X_train, X_test, y_train, y_test, y_pred

    return acc, corr, weights


In [ ]:
output = {}
output["2v2"] = {}
output["pear"] = {}
output["weights"] = {}

for subject in tqdm(subjects):
    output["2v2"][subject] = {}
    output["pear"][subject] = {}
    output["weights"][subject] = {}
    fmri = get_subject_data(subject)

    for roi in ROIS:
        print(f"{subject} {roi}")
        model_outputs = []
        voxels = np.array(fmri[roi])

        for task in TASKS:
            data = np.load(f"./features/pereira_{task}.npy", allow_pickle=True)
            feats = np.array(data[-1])
            with open(f"./models/{task}_{roi}_{subject}.model", "rb") as f:
                model = pickle.load(f)

            model_output = model.predict(feats)
            model_outputs.append(model_output)
            del model, data, feats, model_output

        model_outputs = np.array(model_outputs)
        acc, corr, weights = create_ensemble(model_outputs, voxels)
        output["2v2"][subject][roi] = acc
        output["pear"][subject][roi] = corr
        output["weights"][subject][roi] = weights
        del model_outputs, voxels

    del fmri


In [ ]:
ensemble_output = {}
ensemble_output["2v2"] = {}
ensemble_output["pear"] = {}
ensemble_output["weights"] = {}
for roi in ROIS:
    ensemble_output["2v2"][roi] = {}
    ensemble_output["pear"][roi] = {}
    ensemble_output["weights"][roi] = {}
    for subject in subjects:
        ensemble_output["2v2"][roi][subject] = output["2v2"][subject][roi]
        ensemble_output["pear"][roi][subject] = output["pear"][subject][roi]
        ensemble_output["weights"][roi][subject] = output["weights"][subject][roi]


In [ ]:
with open(f"results/results_ensemble_dyn_wt.pkl", "wb") as f:
    pickle.dump(ensemble_output, f)
